## Adding Language Model to Sunbird-ASR

In [1]:
# Install dependencies
!sudo apt install build-essential cmake libboost-system-dev libboost-thread-dev libboost-program-options-dev libboost-test-dev libeigen3-dev zlib1g-dev libbz2-dev liblzma-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
libboost-program-options-dev is already the newest version (1.65.1.0ubuntu1).
libboost-program-options-dev set to manually installed.
libboost-system-dev is already the newest version (1.65.1.0ubuntu1).
libboost-system-dev set to manually installed.
libboost-thread-dev is already the newest version (1.65.1.0ubuntu1).
libboost-thread-dev set to manually installed.
libboost-test-dev is already the newest version (1.65.1.0ubuntu1).
libboost-test-dev set to manually installed.
cmake is already the newest version (3.10.2-1ubuntu2.18.04.2).
libbz2-dev is already the newest version (1.0.6-8.1ubuntu0.2).
libbz2-dev set to manually installed.
liblzma-dev is already the newest version (5.2.2-1.3ubuntu0.1).
liblzma-dev set to manually installed.
zlib1g-dev is already the newest version (1:1.2.11.dfsg-0ubuntu2.2).
zlib1g-dev set to manually in

In [ ]:
!wget -O - https://kheafield.com/code/kenlm.tar.gz | tar xz

--2022-12-14 02:56:22--  https://kheafield.com/code/kenlm.tar.gz
Resolving kheafield.com (kheafield.com)... 35.196.63.85
Connecting to kheafield.com (kheafield.com)|35.196.63.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 491888 (480K) [application/x-gzip]
Saving to: ‘STDOUT’

-                   100%[===================>] 480.36K  --.-KB/s    in 0.03s   

2022-12-14 02:56:22 (14.1 MB/s) - written to stdout [491888/491888]



In [ ]:
!pip install https://github.com/kpu/kenlm/archive/master.zip pyctcdecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     | 550 kB 1.8 MB/s
     |████████████████████████████████| 45 kB 991 kB/s 
     |████████████████████████████████| 398 kB 11.1 MB/s 
  Created wheel for kenlm: filename=kenlm-0.0.0-cp38-cp38-linux_x86_64.whl size=2396780 sha256=83105195a571862afa28beb5d14c59734d0ef2b3e16609262b6703208ce54399
  Stored in directory: /tmp/pip-ephem-wheel-cache-v3nxk1y4/wheels/ff/08/4e/a3ddc0e786e0f3c1fcd2e7a82c4324c02fc3ae2638471406d2
Successfully built kenlm


A language model that is useful for a speech recognition system should support the acoustic model, e.g. Wav2Vec2, in predicting the next word (or token, letter) and therefore model the following distribution: 

${P}(w_n | \mathbf{w}_0^{t-1})P(w_n​ ∣w 0
t−1 )$  with $w_nw$ n being the next word and ${w}_0^{t-1}w 0
t−1$ being the sequence of all previous words since the beginning of the utterance. Simply said, the language model should be good at predicting the next word given all previously transcribed words regardless of the audio input given to the speech recognition system.



In [ ]:
!pip install datasets >> /dev/null
from datasets import load_dataset

In [ ]:
salt_lg = load_dataset("Sunbird/salt-dataset")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/Sunbird___json/Sunbird--salt-dataset-17f191c1fb7bc902/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/tokens .
    
Token: 
Add token as git credential? (Y/n) y
Token is valid.
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper

In [ ]:
cv_lg = load_dataset("mozilla-foundation/common_voice_11_0", "lg", use_auth_token=True)


Extracting data files #3:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #4:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #2:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

...


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 9787it [00:00, 44485.39it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]




Reading metadata...: 0it [00:00, ?it/s]


Reading metadata...: 5710it [00:00, 57091.34it/s]


Reading metadata...: 13088it [00:00, 66902.65it/s]


Reading metadata...: 19779it [00:00, 66075.12it/s]


Reading metadata...: 26388it [00:00, 63114.19it/s]


Reading metadata...: 38102it [00:00, 64618.81it/s]


Dataset common_voice_11_0 downloaded and prepared to /root/.cache/huggingface/datasets/mozilla-foundation___common_voice_11_0/lg/11.0.0/f8e47235d9b4e68fa24ed71d63266a02018ccf7194b2a8c9c598a5f3ab304d9f. Subsequent calls will reuse this data.


  0%|          | 0/5 [00:00<?, ?it/s]

In [57]:
cv_lg_train = cv_lg.get("train")['sentence']
cv_lg_test = cv_lg.get("test")['sentence']
cv_lg_dev = cv_lg.get("other")['sentence']
cv_lg_inv = cv_lg.get("invalidated")['sentence']

In [58]:
cv_lg_dataset = cv_lg_train + cv_lg_test + cv_lg_dev +cv_lg_inv

In [59]:
len(cv_lg_dataset)

115831

In [60]:
salt_lg = salt_lg.get("train")["Luganda"]

AttributeError: ignored

In [61]:
final_lg_dataset = cv_lg_dataset + salt_lg

In [62]:
len(final_lg_dataset)

140837

In [63]:
chars_to_ignore_regex = '[,?.!\-\;\:"“%‘”�—’…–]'  # change to the ignored characters of your fine-tuned model

In [ ]:
!mkdir kenlm/build && cd kenlm/build && cmake .. && make -j2
!ls kenlm/build/bin

-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Failed
-- Looking for pthread_create in pthreads
-- Looking for pthread_create in pthreads - not found
-- Looking for pthread_create in pthread
-- Looking for pthread_create in pthread - found
-- Found Threads: TRUE  
-- Found Boost: /usr/include (found suitable version "1.65.1", minimum required is "1.41.0") found components: program_options system

In [141]:
chars_to_ignore_regex = '[,?.!\-\;\:"“%‘”�—’…–]'  # change to the ignored characters of your fine-tuned model

In [142]:
import re

def extract_text(batch):
  text = batch
  text = re.sub(chars_to_ignore_regex, "", text.lower())
  return text

In [143]:
v = [extract_text(i) for i in final_lg_dataset]

In [144]:
v[9]

"ebyo eby'okuzimba binajja nga tumaze okuzaala"

In [145]:
final_lg_dataset[9]

"Ebyo eby'okuzimba binajja nga tumaze okuzaala."

In [ ]:
v

In [147]:
with open("text.txt", "w") as file:
  file.write(" ".join(v))

In [148]:
!kenlm/build/bin/lmplz -o 5 <"text.txt" > "5gram.arpa"

=== 1/5 Counting and sorting n-grams ===
Reading /content/text.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
tcmalloc: large alloc 1917870080 bytes == 0x55792944e000 @  0x7f25322001e7 0x5579276ea7e2 0x5579276854fe 0x5579276642eb 0x557927650066 0x7f2530399c87 0x557927651baa
tcmalloc: large alloc 8950054912 bytes == 0x55799b954000 @  0x7f25322001e7 0x5579276ea7e2 0x5579276d980a 0x5579276da248 0x557927664308 0x557927650066 0x7f2530399c87 0x557927651baa
****************************************************************************************************
Unigram tokens 1081686 types 106167
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:1274004 2:1062616064 3:1992405248 4:3187848192 5:4648945664
tcmalloc: large alloc 4648951808 bytes == 0x55792944e000 @  0x7f25322001e7 0x5579276ea7e2 0x5579276d980a 0x5579276da248 0x5579276648d7 0x557927650066 0x7f2530399c87 0x557927651baa
tcmalloc: large alloc 1992409088 bytes == 0

In [149]:
!head -20 5gram.arpa

\data\
ngram 1=106166
ngram 2=522330
ngram 3=766140
ngram 4=858286
ngram 5=910518

\1-grams:
-5.7587533	<unk>	0
0	<s>	-0.077299215
-2.2717988	era	-0.27781975
-5.6349707	yayokebwa	-0.077299215
-5.6349707	bwokebwa	-0.077299215
-4.121536	ekkolero	-0.29917124
-5.3229795	erikola	-0.077299215
-4.906173	sabbuuni	-0.077299215
-4.1931214	lijja	-0.107203774
-5.1759996	kuteekebwawo	-0.077299215
-1.6308211	mu	-0.8067609
-5.4599195	kibangirizi	-0.077299215


In [150]:
with open("5gram.arpa", "r") as read_file, open("5gram_correct.arpa", "w") as write_file:
  has_added_eos = False
  for line in read_file:
    if not has_added_eos and "ngram 1=" in line:
      count=line.strip().split("=")[-1]
      write_file.write(line.replace(f"{count}", f"{int(count)+1}"))
    elif not has_added_eos and "<s>" in line:
      write_file.write(line)
      write_file.write(line.replace("<s>", "</s>"))
      has_added_eos = True
    else:
      write_file.write(line)

In [151]:
!head -20 5gram_correct.arpa

\data\
ngram 1=106167
ngram 2=522330
ngram 3=766140
ngram 4=858286
ngram 5=910518

\1-grams:
-5.7587533	<unk>	0
0	<s>	-0.077299215
0	</s>	-0.077299215
-2.2717988	era	-0.27781975
-5.6349707	yayokebwa	-0.077299215
-5.6349707	bwokebwa	-0.077299215
-4.121536	ekkolero	-0.29917124
-5.3229795	erikola	-0.077299215
-4.906173	sabbuuni	-0.077299215
-4.1931214	lijja	-0.107203774
-5.1759996	kuteekebwawo	-0.077299215
-1.6308211	mu	-0.8067609


In [ ]:
!pip install transformers >> /dev/null
# from transformers import AutoProcessor

# processor = AutoProcessor.from_pretrained("hf-test/xls-r-300m-sv")

In [152]:
from transformers import AutoProcessor
processor = AutoProcessor.from_pretrained("Sunbird/sunbird-asr")

/usr/local/lib/python3.8/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in 'snapshot_download': allow_regex. Will not be supported from version '0.12'.

Please use `allow_patterns` and `ignore_patterns` instead.
  warnings.warn(message, FutureWarning)


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

In [153]:
vocab_dict = processor.tokenizer.get_vocab()
sorted_vocab_dict = {k.lower(): v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1])}

In [154]:
vocab_dict

{'[PAD]': 49,
 '[UNK]': 48,
 '\\': 1,
 ']': 2,
 '`': 3,
 'a': 4,
 'b': 5,
 'c': 6,
 'd': 7,
 'e': 8,
 'f': 9,
 'g': 10,
 'h': 11,
 'i': 12,
 'j': 13,
 'k': 14,
 'l': 15,
 'm': 16,
 'n': 17,
 'o': 18,
 'p': 19,
 'q': 20,
 'r': 21,
 's': 22,
 't': 23,
 'u': 24,
 'v': 25,
 'w': 26,
 'x': 27,
 'y': 28,
 'z': 29,
 '|': 0,
 '½': 30,
 'á': 31,
 'é': 32,
 'ó': 33,
 'ŋ': 34,
 'š': 35,
 'η': 36,
 'ղ': 37,
 '῎': 38,
 '῝': 39,
 '`': 40,
 '–': 41,
 '—': 42,
 '“': 43,
 '”': 44,
 '‟': 45,
 '•': 46,
 '…': 47,
 '<s>': 50,
 '</s>': 51}

In [160]:
from pyctcdecode import build_ctcdecoder

decoder = build_ctcdecoder(
    labels=list(sorted_vocab_dict.keys()),
    kenlm_model_path="5gram_correct.arpa",
)

In [156]:
from transformers import Wav2Vec2ProcessorWithLM

processor_with_lm = Wav2Vec2ProcessorWithLM(
    feature_extractor=processor.feature_extractor,
    tokenizer=processor.tokenizer,
    decoder=decoder
)

In [158]:
processor_with_lm

Wav2Vec2ProcessorWithLM:
- feature_extractor: Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "processor_class": "Wav2Vec2ProcessorWithLM",
  "return_attention_mask": true,
  "sampling_rate": 16000
}

- tokenizer: PreTrainedTokenizer(name_or_path='Sunbird/sunbird-asr', vocab_size=50, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[UNK]', 'pad_token': '[PAD]', 'additional_special_tokens': [AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True)]})

In [ ]:
!sudo apt-get install git-lfs tree

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 20 not upgraded.
Need to get 40.7 kB of archives.
After this operation, 105 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tree amd64 1.7.0-5 [40.7 kB]
Fetched 40.7 kB in 0s (198 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falli

In [ ]:
from huggingface_hub import Repository
repo = Repository(local_dir="sunbird-asr", clone_from="Sunbird/sunbird-asr")

Cloning https://huggingface.co/Sunbird/sunbird-asr into local empty directory.


Download file optimizer.pt:   0%|          | 3.44k/2.32G [00:00<?, ?B/s]

Download file scheduler.pt: 100%|##########| 623/623 [00:00<?, ?B/s]

Download file rng_state_0.pth:  25%|##4       | 3.48k/14.2k [00:00<?, ?B/s]

Download file rng_state_3.pth:  24%|##4       | 3.48k/14.2k [00:00<?, ?B/s]

...

Clean file training_args.bin:  34%|###4      | 1.00k/2.92k [00:00<?, ?B/s]

Download file pytorch_model.bin:   0%|          | 1.43k/1.18G [00:00<?, ?B/s]

Clean file scaler.pt: 100%|##########| 559/559 [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/1.18G [00:00<?, ?B/s]

Clean file optimizer.pt:   0%|          | 1.00k/2.32G [00:00<?, ?B/s]

In [109]:
processor_with_lm

Wav2Vec2ProcessorWithLM:
- feature_extractor: Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "processor_class": "Wav2Vec2ProcessorWithLM",
  "return_attention_mask": true,
  "sampling_rate": 16000
}

- tokenizer: PreTrainedTokenizer(name_or_path='Sunbird/sunbird-asr', vocab_size=50, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[UNK]', 'pad_token': '[PAD]', 'additional_special_tokens': [AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True)]})

In [110]:
vocab_dict = processor_with_lm.tokenizer.get_vocab()
sorted_vocab_dict = {k.lower(): v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1])}

In [111]:
sorted_vocab_dict

{'|': 0,
 '\\': 1,
 ']': 2,
 '`': 3,
 'a': 4,
 'b': 5,
 'c': 6,
 'd': 7,
 'e': 8,
 'f': 9,
 'g': 10,
 'h': 11,
 'i': 12,
 'j': 13,
 'k': 14,
 'l': 15,
 'm': 16,
 'n': 17,
 'o': 18,
 'p': 19,
 'q': 20,
 'r': 21,
 's': 22,
 't': 23,
 'u': 24,
 'v': 25,
 'w': 26,
 'x': 27,
 'y': 28,
 'z': 29,
 '½': 30,
 'á': 31,
 'é': 32,
 'ó': 33,
 'ŋ': 34,
 'š': 35,
 'η': 36,
 'ղ': 37,
 '῎': 38,
 '῝': 39,
 '`': 40,
 '–': 41,
 '—': 42,
 '“': 43,
 '”': 44,
 '‟': 45,
 '•': 46,
 '…': 47,
 '[unk]': 48,
 '[pad]': 49,
 '<s>': 50,
 '</s>': 51}

In [107]:
!tree -h sunbird-asr/

sunbird-asr/
├── [  30]  added_tokens.json
├── [ 389]  alphabet.json
├── [2.3K]  config.json
├── [4.0K]  language_model
│   ├── [ 71M]  5gram.bin
│   ├── [  78]  attrs.json
│   └── [1.5M]  unigrams.txt
├── [2.3G]  optimizer.pt
├── [ 262]  preprocessor_config.json
├── [1.2G]  pytorch_model.bin
├── [ 14K]  rng_state_0.pth
├── [ 14K]  rng_state_1.pth
├── [ 14K]  rng_state_2.pth
├── [ 14K]  rng_state_3.pth
├── [ 559]  scaler.pt
├── [ 623]  scheduler.pt
├── [ 679]  special_tokens_map.json
├── [ 575]  tokenizer_config.json
├── [ 52K]  trainer_state.json
├── [2.9K]  training_args.bin
└── [ 580]  vocab.json

1 directory, 20 files


In [108]:
# Reduce the size of the file 
!kenlm/build/bin/build_binary sunbird-asr/language_model/5gram_correct.arpa sunbird-asr/language_model/5gram.bin


/content/kenlm/util/file.cc:76 in int util::OpenReadOrThrow(const char*) threw ErrnoException because `-1 == (ret = open(name, 00))'.
No such file or directory while opening sunbird-asr/language_model/5gram_correct.arpa
ERROR


In [ ]:
!rm sunbird-asr/language_model/5gram_correct.arpa && tree -h sunbird-asr/

sunbird-asr/
├── [  30]  added_tokens.json
├── [ 389]  alphabet.json
├── [2.3K]  config.json
├── [4.0K]  language_model
│   ├── [ 71M]  5gram.bin
│   ├── [  78]  attrs.json
│   └── [1.5M]  unigrams.txt
├── [2.3G]  optimizer.pt
├── [ 262]  preprocessor_config.json
├── [1.2G]  pytorch_model.bin
├── [ 14K]  rng_state_0.pth
├── [ 14K]  rng_state_1.pth
├── [ 14K]  rng_state_2.pth
├── [ 14K]  rng_state_3.pth
├── [ 559]  scaler.pt
├── [ 623]  scheduler.pt
├── [ 406]  special_tokens_map.json
├── [ 545]  tokenizer_config.json
├── [ 52K]  trainer_state.json
├── [2.9K]  training_args.bin
└── [ 580]  vocab.json

1 directory, 20 files


In [ ]:
repo.push_to_hub(commit_message="Upload lm-boosted decoder")

Upload file language_model/5gram.bin:   0%|          | 3.30k/71.3M [00:00<?, ?B/s]

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/Sunbird/sunbird-asr
   086129f..9011399  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/Sunbird/sunbird-asr
   086129f..9011399  main -> main



'https://huggingface.co/Sunbird/sunbird-asr/commit/90113991aa2c4200698e477c795f435ed49480fc'

In [ ]:
from transformers import Wav2Vec2ProcessorWithLM
processor = Wav2Vec2ProcessorWithLM.from_pretrained("sunbird-asr")


In [ ]:
transcription = processor.batch_decode(logits.numpy()).text
transcription[0].lower()

NameError: ignored